# **Cart-Pole System and Implementation in Crocoddyl**

This notebook implements the cartpole system using **Crocoddyl**, a differential dynamic programming solver for robotic control. The key features of this notebook are
 - **System Modeling**: Define the cartpole dynamics as an action model in Crocoddyl.
 - **Cost Function**: Implement a cost function that penalizes deviations from the upright position and large control inputs within an action model.
 - **Derivatives Computation**: Compute analytical derivatives of the dynamics and cost function for efficient optimization.
 - **Numerical differentiation**: Validate analytical derivatives using Crocoddyl's `ActionModelNumDiff` class.
 - **Simulation & Optimization**: Simulate the system and solves an optimal control problem using Crocoddyl.

## Overview of the Cart-Pole System

The cart-pole system consists of a cart that can move along a horizontal track and a pole attached to the cart via a pivot joint. The goal is to balance the pole in the upright position by applying forces to the cart.

<div style="text-align:center;">
    <img src="https://www.ashwinnarayan.com/img/cart-pole.png" width="300" height="300">
</div>

## Equations of Motion

The state of the system is given by:

- $y$: Position of the cart
- $\theta$: Angle of the pole from the upright position
- $\dot{y}$: Velocity of the cart
- $\dot{\theta}$: Angular velocity of the pole

The equations of motion are derived from Newtonian mechanics:

\begin{aligned}
\mu &= m_{\text{cart}} + m_{\text{pole}} \sin^2(\theta) \\
\ddot{y} &= \frac{f}{\mu} + \frac{m_{\text{pole}} g \cos(\theta) \sin(\theta)}{\mu} - \frac{m_{\text{pole}} l_{\text{pole}} \sin(\theta) \dot{\theta}^2}{\mu} \\
\ddot{\theta} &= \frac{f \cos(\theta)}{l_{\text{pole}} \mu} + \frac{(m_{\text{cart}} + m_{\text{pole}}) g \sin(\theta)}{l_{\text{pole}} \mu} - \frac{m_{\text{pole}} \cos(\theta) \sin(\theta) \dot{\theta}^2}{\mu}
\end{aligned}

where:
 - $\ddot{y}$, $\ddot{\theta}$ are accelerations
 - $f$ is the control force applied to the cart
 - $m_{\text{cart}}$ is the mass of the cart
 - $m_{\text{pole}}$ is the mass of the pole
 - $l_{\text{pole}}$ is the length of the pole
 - $g$ is the gravitational acceleration

The system dynamics update is given by:

\begin{aligned}
y_{t+1} &= y_t + \Delta t\, \dot{y}_{t+1} \\
\theta_{t+1} &= \theta_t + \Delta t\, \dot{\theta}_{t+1} \\
\dot{y}_{t+1} &= \dot{y}_t + \Delta t\, \ddot{y} \\
\dot{\theta}_{t+1} &= \dot{\theta}_t + \Delta t\, \ddot{\theta}
\end{aligned}

### Cost Function
A quadratic cost function is used to regulate the system:

\begin{equation}
J = \frac{1}{2} \sum w_i r_i^2
\end{equation}

where residuals $r$ are:

\begin{equation}
r = \begin{bmatrix} \sin \theta, 1 - \cos \theta, y, \dot{y}, \dot{\theta}, f \end{bmatrix}
\end{equation}

and weights $w$ penalize deviations.


## Implementation in Crocoddyl

### ActionModelCartpole

This class models the cart-pole system using Crocoddyl’s `ActionModelAbstract`. It defines the system dynamics and cost function.

Attributes:
 - `Δt`: Time step for integration.
 - `m_cart`: Mass of the cart.
 - `m_pole`: Mass of the pole.
 - `l_pole`: Length of the pole.
 - `grav`: Gravitational acceleration.
 - `costWeights`: Weights for the cost function.

Methods:
 - `calc(data, x, u)`: Computes the system dynamics and cost.
 - `calcDiff(data, x, u)`: Computes the derivatives of the system dynamics and cost function.
 - `createData()`: Creates an instance of `ActionDataCartpole`.

Below, you can see

In [ ]:
import crocoddyl
import numpy as np

class ActionModelCartpole(crocoddyl.ActionModelAbstract):
    """
    Defines the action model for the cart-pole system in Crocoddyl.
    
    This model simulates a cart moving along a horizontal track with a 
    freely rotating pole attached to it. The goal is to control the force 
    applied to the cart to stabilize the pole.
    
    State vector: [y, θ, ẏ, θ̇]
    Control input: [f] (force applied to the cart)
    """
    def __init__(self):
        """
        Initializes the cart-pole action model with system parameters and cost weights.
        """
        super().__init__(crocoddyl.StateVector(4), 1, 6)  # nu = 1; nr = 6

        self.Δt = 5e-2
        self.m_cart = 1.0
        self.m_pole = 0.1
        self.l_pole = 0.5
        self.grav = 9.81
        self.costWeights = [
            1.0,
            1.0,
            0.1,
            0.001,
            0.001,
            1.0,
        ]  # sin, 1-cos, x, xdot, thdot, f

    def calc(self, data, x, u=None):
        """
        Computes the next state and the cost function.

        Args:
            data (ActionDataCartpole): Data structure to store intermediate results.
            x (numpy.ndarray): State vector [y, θ, ẏ, θ̇].
            u (numpy.ndarray, optional): Control input [f]. Defaults to None.

        Updates:
            - Next state xnext.
            - Cost residuals r.
            - Total cost value.
        """
        if u is not None:
            # Getting the state and control variables
            y, θ, ẏ, θ̇, f = x[0], x[1], x[2], x[3], u[0]
            # Shortname for system parameters
            Δt, m_cart, m_pole, l_pole, grav, w = self.Δt, self.m_cart, self.m_pole, self.l_pole, self.grav, self.costWeights
            sin_θ, cos_θ = np.sin(θ), np.cos(θ)
            # Computing the cartpole dynamics
            data.μ = m_cart + m_pole * sin_θ * sin_θ
            data.ÿ̈ = f / data.μ
            data.ÿ̈ += m_pole * grav * cos_θ * sin_θ / data.μ
            data.ÿ̈ -= m_pole * l_pole * (sin_θ * θ̇ ** 2 / data.μ)
            data.θ̈ = (f / l_pole) * cos_θ / data.μ
            data.θ̈ += ((m_cart + m_pole) * grav / l_pole) * sin_θ / data.μ
            data.θ̈ -= m_pole * cos_θ * sin_θ * θ̇**2 / data.μ
            data.ẏ_next = ẏ + Δt * data.ÿ̈
            data.θ̇_next = θ̇ + Δt * data.θ̈
            data.y_next = y + Δt * data.ẏ_next
            data.θ_next = θ + Δt * data.θ̇_next
            data.xnext[:] = np.array([data.y_next, data.θ_next, data.ẏ_next, data.θ̇_next])
            # Computing the cost residual and value
            data.r[:] = w * np.array([sin_θ, 1.0 - cos_θ, y, ẏ, θ̇, f])
            data.cost = 0.5 * sum(data.r ** 2)
        else:
            # Getting the state and control variables
            y, θ, ẏ, θ̇ = x[0], x[1], x[2], x[3]
            w = self.costWeights
            sin_θ, cos_θ = np.sin(θ), np.cos(θ)
            data.xnext[:] = x
            # Computing the cost residual and value
            data.r[:] = w * np.array([sin_θ, 1.0 - cos_θ, y, ẏ, θ̇, 0.0])
            data.cost = 0.5 * sum(data.r ** 2)

    def calcDiff(self, data, x, u=None):
        """
        Computes the derivatives of the dynamics and the cost function.

        Args:
            data (ActionDataCartpole): Data structure to store intermediate results.
            x (numpy.ndarray): State vector [y, θ, ẏ, θ̇].
            u (numpy.ndarray, optional): Control input [f]. Defaults to None.

        Updates:
            - Derivetives of the dynamics.
            - Derivatives of the cost function.
        """
        if u is not None:
            # Getting the state and control variables
            y, θ, ẏ, θ̇, f = x[0], x[1], x[2], x[3], u[0]
            # Shortname for system parameters
            Δt, m_cart, m_pole, l_pole, grav, w = self.Δt, self.m_cart, self.m_pole, self.l_pole, self.grav, self.costWeights
            sin_θ, cos_θ = np.sin(θ), np.cos(θ)
            # Computing the derivative of the cartpole dynamics
            data.dμ_dθ = 2.0 * m_pole * sin_θ * cos_θ
            data.dÿ̈_dy = 0.0
            data.dÿ̈_dθ = m_pole * grav * (cos_θ**2 - sin_θ**2) / data.μ
            data.dÿ̈_dθ -= m_pole * l_pole * cos_θ * θ̇ * θ̇ / data.μ
            data.dÿ̈_dθ -= data.dμ_dθ * data.ÿ̈ / data.μ
            data.dÿ̈_dẏ = 0.0
            data.dÿ̈_dθ̇ = -2.0 * m_pole * l_pole * sin_θ * θ̇ / data.μ
            data.dÿ̈_du = 1.0 / data.μ
            data.dθ̈_dy = 0.0
            data.dθ̈_dθ = -(f / l_pole) * sin_θ / data.μ
            data.dθ̈_dθ += ((m_cart + m_pole) * grav / l_pole) * cos_θ / data.μ
            data.dθ̈_dθ -= m_pole * (cos_θ**2 - sin_θ**2) * θ̇ * θ̇ / data.μ
            data.dθ̈_dθ -= data.dμ_dθ * data.θ̈ / data.μ
            data.dθ̈_dẏ = 0.0
            data.dθ̈_dθ̇ = -2.0 * m_pole * cos_θ * sin_θ * θ̇ / data.μ
            data.dθ̈_du = cos_θ / (l_pole * data.μ)
            data.dẏ_next_dy = Δt * data.dÿ̈_dy
            data.dẏ_next_dθ = Δt * data.dÿ̈_dθ
            data.dẏ_next_dẏ = 1.0 + Δt * data.dÿ̈_dẏ
            data.dẏ_next_dθ̇ = Δt * data.dÿ̈_dθ̇
            data.dẏ_next_du = Δt * data.dÿ̈_du
            data.dθ̇_next_dy = Δt * data.dθ̈_dy 
            data.dθ̇_next_dθ = Δt * data.dθ̈_dθ
            data.dθ̇_next_dẏ = Δt * data.dθ̈_dẏ
            data.dθ̇_next_dθ̇ = 1.0 + Δt * data.dθ̈_dθ̇
            data.dθ̇_next_du = Δt * data.dθ̈_du
            data.dy_next_dy = 1.0 + Δt * data.dẏ_next_dy
            data.dy_next_dθ = Δt * data.dẏ_next_dθ
            data.dy_next_dẏ = Δt * data.dẏ_next_dẏ
            data.dy_next_dθ̇ = Δt * data.dẏ_next_dθ̇
            data.dy_next_du = Δt * data.dẏ_next_du
            data.dθ_next_dy = Δt * data.dθ̇_next_dy
            data.dθ_next_dθ = 1.0 + Δt * data.dθ̇_next_dθ
            data.dθ_next_dẏ = Δt * data.dθ̇_next_dẏ
            data.dθ_next_dθ̇ = Δt * data.dθ̇_next_dθ̇
            data.dθ_next_du = Δt * data.dθ̇_next_du
            data.Fx[:, :] = np.array([[data.dy_next_dy, data.dy_next_dθ, data.dy_next_dẏ, data.dy_next_dθ̇],
                                      [data.dθ_next_dy, data.dθ_next_dθ, data.dθ_next_dẏ, data.dθ_next_dθ̇],
                                      [data.dẏ_next_dy, data.dẏ_next_dθ, data.dẏ_next_dẏ, data.dẏ_next_dθ̇],
                                      [data.dθ̇_next_dy, data.dθ̇_next_dθ, data.dθ̇_next_dẏ, data.dθ̇_next_dθ̇]])
            data.Fu[:] = np.array([data.dy_next_du, data.dθ_next_du, data.dẏ_next_du, data.dθ̇_next_du])
            # Computing derivatives of the cost function
            w0_2, w1_2, w2_2, w3_2, w4_2, w5_2 = w[0] * w[0], w[1] * w[1], w[2] * w[2], w[3] * w[3], w[4] * w[4], w[5] * w[5]
            data.Lx[0] = w2_2 * y
            data.Lx[1] = w0_2 * sin_θ * cos_θ + w1_2 * (1.0 - cos_θ) * sin_θ
            data.Lx[2] = w3_2 * ẏ
            data.Lx[3] = w4_2 * θ̇
            data.Lu[0] = w5_2 * f
            data.Lxx[0, 0] = w2_2
            data.Lxx[1, 1] = w0_2 * (cos_θ * cos_θ - sin_θ * sin_θ)
            data.Lxx[1, 1] += w1_2 * ((1.0 - cos_θ) * cos_θ + sin_θ * sin_θ)
            data.Lxx[2, 2] = w3_2
            data.Lxx[3, 3] = w4_2
            data.Luu[:] = w5_2
        else:
            # Getting the state and control variables
            y, θ, ẏ, θ̇ = x[0], x[1], x[2], x[3]
            w = self.costWeights
            sin_θ, cos_θ = np.sin(θ), np.cos(θ)
            # Computing the derivative of the cartpole dynamics
            for i in range(self.state.ndx):
                data.Fx[i, i] = 1.0
            # Computing derivatives of the cost function
            w0_2, w1_2, w2_2, w3_2, w4_2, w5_2 = w[0] * w[0], w[1] * w[1], w[2] * w[2], w[3] * w[3], w[4] * w[4], w[5] * w[5]
            data.Lx[0] = w2_2 * y
            data.Lx[1] = w0_2 * sin_θ * cos_θ + w1_2 * (1.0 - cos_θ) * sin_θ
            data.Lx[2] = w3_2 * ẏ
            data.Lx[3] = w4_2 * θ̇
            data.Lxx[0, 0] = w2_2
            data.Lxx[1, 1] = w0_2 * (cos_θ * cos_θ - sin_θ * sin_θ)
            data.Lxx[1, 1] += w1_2 * ((1.0 - cos_θ) * cos_θ + sin_θ * sin_θ)
            data.Lxx[2, 2] = w3_2
            data.Lxx[3, 3] = w4_2

    def createData(self):
        """
        Creates the action data structure for the cart-pole model.

        Returns:
            ActionDataCartpole: Data structure to store intermediate computations.
        """
        return ActionDataCartpole(self)


class ActionDataCartpole(crocoddyl.ActionDataAbstract):
    """
    Data structure for storing intermediate computations of the cart-pole dynamics.
    """
    def __init__(self, model):
        """
        Initializes the data structure with default values.

        Args:
            model (ActionModelCartpole): Cart-pole action model.
        """
        super().__init__(model)
        self.μ = 0.0
        self.ÿ̈ = 0.0
        self.θ̈ = 0.0
        self.ẏ_next = 0.0
        self.θ̇_next = 0.0
        self.y_next = 0.0
        self.θ_next = 0.0
        self.dμ_dθ = 0.0
        self.dÿ̈_dy = 0.0
        self.dÿ̈_dθ = 0.0
        self.dÿ̈_dθ = 0.0
        self.dÿ̈_dθ = 0.0
        self.dÿ̈_dẏ = 0.0
        self.dÿ̈_dθ̇ = 0.0
        self.dÿ̈_du = 0.0
        self.dθ̈_dy = 0.0
        self.dθ̈_dθ = 0.0
        self.dθ̈_dθ = 0.0
        self.dθ̈_dθ = 0.0
        self.dθ̈_dθ = 0.0
        self.dθ̈_dẏ = 0.0
        self.dθ̈_dθ̇ = 0.0
        self.dẏ_next_dy = 0.0
        self.dẏ_next_dθ = 0.0
        self.dẏ_next_dẏ = 0.0
        self.dẏ_next_dθ̇ = 0.0
        self.dẏ_next_du = 0.0
        self.dθ̇_next_dy = 0.0
        self.dθ̇_next_dθ = 0.0
        self.dθ̇_next_dẏ = 0.0
        self.dθ̇_next_dθ̇ = 0.0
        self.dθ̇_next_du = 0.0
        self.dy_next_dy = 0.0
        self.dy_next_dθ = 0.0
        self.dy_next_dẏ = 0.0
        self.dy_next_dθ̇ = 0.0
        self.dy_next_du = 0.0
        self.dθ_next_dy = 0.0
        self.dθ_next_dθ = 0.0
        self.dθ_next_dẏ = 0.0
        self.dθ_next_dθ̇ = 0.0
        self.dθ_next_du = 0.0

## Validating Analytical Derivatives with Numerical Differentiation

After defining the action model and deriving the analytical derivatives of the system’s dynamics and cost function, it is important to verify the correctness of these derivatives. One common method for validating derivatives is **numerical differentiation (NumDiff)**. This method approximates the derivative of a function by comparing the values of the function at small increments around a given point.

The principle behind numerical differentiation is the approximation of the derivative of a function \( f(x) \) using the following formula:

\begin{equation}
\frac{df(x)}{dx} \approx \frac{f(x + \epsilon) - f(x)}{\epsilon}
\end{equation}

where:
 - $\epsilon$ is a small perturbation value (often called the step size), which controls the precision of the derivative approximation.
 - $f(x)$ is the function whose derivative is being approximated.

In Crocoddyl, we perform numerical differentiation of an action model via `ActionModelNumDiff` class. This class is designed to define $\epsilon$ values appropriately. This is done by consider the type of derivatives and floating-point number (`double`, `float`). Below, we show how to use for asserting our analytical derivatives.

In [ ]:
# Create an action model, data, and its NumDiff's action
model = ActionModelCartpole()
data = model.createData()
model_nd = crocoddyl.ActionModelNumDiff(model)
data_nd = model_nd.createData()

# Generate a random state and control
x = model.state.rand()
u = np.random.rand(1)

# Compute analytical and NumDiff derivatives
model.calc(data, x, u)
model.calcDiff(data, x, u)
model_nd.calc(data_nd, x, u)
model_nd.calcDiff(data_nd, x, u)

# Compare derivatives
print("Fx difference:", np.linalg.norm(data.Fx - data_nd.Fx))
print("Fu difference:", np.linalg.norm(data.Fu - data_nd.Fu))
print("Lx difference:", np.linalg.norm(data.Lx - data_nd.Lx))
print("Lu difference:", np.linalg.norm(data.Lu - data_nd.Lu))
print("Lxx difference:", np.linalg.norm(data.Lxx - data_nd.Lxx))
print("Lxu difference:", np.linalg.norm(data.Lxu - data_nd.Lxu))
print("epsilon", model_nd.disturbance)

## Simulating our Cart-Pole System

After we have verified the analytical derivatives, we want to analyse/validate the cart-pole dynamics. To do so, we rollout the system dynamics without applying control commands. In Crocoddyl, this can be done easily using our `ShootingProblem` class.

In [ ]:
# Create a shooting problem with 50 running nodes (i.e., 50 * 5e-2 = 2.5 sec)
N = 50
x0 = np.array([0.0, 0.0, 1.0, 0.5])
problem = crocoddyl.ShootingProblem(x0, [model] * N, model)

xs = problem.rollout([np.zeros(model.nu)] * N)

In `cartpole_utils`, we provide two functions for plotting and animating our cart-pole. These functions are `plotCartpole` and `animateCartpole`.

In [ ]:
from IPython.display import HTML
from cartpole_utils import animateCartpole

anim = animateCartpole(xs)

# HTML(anim.to_jshtml())
HTML(anim.to_html5_video())

# If you encounter problems probably you need to install ffmpeg/libav-tools:
# sudo apt-get install ffmpeg
# or
# sudo apt-get install libav-tools
# or
# conda install ffmpeg
# Additionally, you need to install matplolib.

As expected, the cart is moving to the left, and the pole is falling down. This behavior occurs because the initial velocities of both the cart and the pole are non-zero. To stabilize the system and find an optimal trajectory for balancing the cart-pole, we formulate a nonlinear optimal control problem as follows:

\begin{aligned}
& \underset{u(t)}{\text{minimize}} \quad J = \frac{1}{2} \sum_{i=0}^{N} w_i \, r_i^2(\mathbf{x}, \mathbf{u})  \\
& \text{subject to} \quad \mathbf{x}^+ = f(\mathbf{x}, \mathbf{u})
\end{aligned}

Where:
 - $J$ is the cost function, which sums over the trajectory nodes  i  the weighted square of the residual  r_i(\mathbf{x}, \mathbf{u}) , capturing the deviation from the desired state and control input.
 - $w_i$ is the weight associated with each node in the trajectory, typically representing how much importance we assign to a given time step in the optimization.
 - $\mathbf{x}^+$  represents the state at the next time step, while $\mathbf{x}$ is the current state.
 - $f(\mathbf{x}, \mathbf{u})$ represents the system’s dynamics, describing how the state evolves from one time step to the next under the control input $\mathbf{u}[i]$.

This formulation defines the cost and dynamics of each node $i$ based on the cart-pole’s action model. All of these elements are encapsulated within Crocoddyl’s shooting problem framework, where the optimization problem is structured as a trajectory optimization problem.

With this setup, we now have all the necessary components to solve the problem using our feasibility-driven Differential Dynamic Programming (DDP) solver. The DDP solver will iteratively adjust the control inputs $\mathbf{u}[i]$ to minimize the cost function while ensuring the system adheres to its dynamics, ultimately finding an optimal trajectory that stabilizes the cart-pole system.

In [ ]:
# Creating the FDDP solver and setting the logger callback
solver = crocoddyl.SolverFDDP(problem)
solver.setCallbacks([crocoddyl.CallbackVerbose()])

# Solving this problem and display the optimal trajectory of our cart-pole system
solver.solve()
anim = animateCartpole(solver.xs)

# HTML(anim.to_jshtml())
HTML(anim.to_html5_video())

We can observe how the **Feasibily-Driven Differential Dynamic Programming (FDDP)** algorithm iteratively finds the optimal trajectory and control sequence. Additionally, the logger provides valuable insights at each iteration, displaying key quantities such as the cost, gradient, infeasibilities, and other relevant metrics. These quantities help us understand the numerical behavior of the optimization process, including convergence patterns and any potential issues during the search for the optimal solution.

## What's Next?

Now that we’ve explored the optimal control problem and seen how FDDP can be applied to find optimal trajectories and control sequences, the next step is to understand how action models can be code-generated for various systems. In this section, we will walk through:
 1. **Action Model Code Generation**: Learn how to automatically generate the action model for cart-pole and other robotic systems using the `ActionModelCodeGen` class.
 2. **Derivatives and Sensitivity Analysis**: Understand how to validate the action models by comparing the code-generated derivatives with numerical derivatives generated using tools like NumDiff.
 3. **Inspecting Code-Generated Performance**: Familiarize yourself with Crocoddyl’s profiling tools to analyze the performance of the generated action models and evaluate computation efficiency.
 4. **Scaling to Different Systems**: Once you’re familiar with the cart-pole system, we will show how to extend the code generation process to more complex robotic systems, enabling efficient computation and optimization.

By the end of the `05_codegenerating_a_cartpole.ipynb`, you’ll be equipped to generate and validate action models for your own robotics applications and simulations, improving the efficiency and accuracy of your control systems. 🚀